In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
count=0
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        count += 1
        if count >= 10:
            break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/.DS_Store
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/.DS_Store
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1947_bacteria_4876.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1946_bacteria_4875.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1952_bacteria_4883.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1954_bacteria_4886.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1951_bacteria_4882.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1949_bacteria_4880.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/.DS_Store
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/NORMAL/NORMAL

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, Input, Concatenate, Layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
import cv2
import os
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf

2025-06-05 09:03:15.300784: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749114195.515417      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749114195.577960      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
labels = ['PNEUMONIA', 'NORMAL']
def get_training_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (150, 150))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data, dtype=object)

In [4]:
train = get_training_data('/kaggle/input/chest-xray-pneumonia/chest_xray/train')
test = get_training_data('/kaggle/input/chest-xray-pneumonia/chest_xray/test')

In [5]:
x_train = []
y_train = []
for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

x_test = []
y_test = []
for feature, label in test:
    x_test.append(feature)
    y_test.append(label)

x_train = np.array(x_train) / 255.0
x_test = np.array(x_test) / 255.0

img_size = 150
x_train = x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_test = x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

In [6]:

data_generator = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=30,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
data_generator.fit(x_train)


In [7]:
def extract_patches(image, patch_size=50, stride=25):  # Reduced stride for overlapping patches
    patches = []
    height, width, channels = image.shape
    for i in range(0, height - patch_size + 1, stride):
        for j in range(0, width - patch_size + 1, stride):
            patch = image[i:i+patch_size, j:j+patch_size, :]
            patches.append(patch)
    return np.array(patches)


patch_size = 50
stride = 25  # Overlapping patches
num_patches_per_image = ((150 - patch_size) // stride + 1) * ((150 - patch_size) // stride + 1)  # 5x5 = 25 patches per image


x_train_patches = []
y_train_patches = []
for img, label in zip(x_train, y_train):
    patches = extract_patches(img, patch_size, stride)
    x_train_patches.extend(patches)
    y_train_patches.extend([label] * len(patches))


x_test_patches = []
y_test_patches = []
for img, label in zip(x_test, y_test):
    patches = extract_patches(img, patch_size, stride)
    x_test_patches.extend(patches)
    y_test_patches.extend([label] * len(patches))

x_train_patches = np.array(x_train_patches)
y_train_patches = np.array(y_train_patches)
x_test_patches = np.array(x_test_patches)
y_test_patches = np.array(y_test_patches)

print(f"Total training patches: {x_train_patches.shape}")
print(f"Total test patches: {x_test_patches.shape}")

Total training patches: (130400, 50, 50, 1)
Total test patches: (15600, 50, 50, 1)


In [8]:
patch_inputs = Input(shape=(patch_size, patch_size, 1))
x = Conv2D(64, (3, 3), strides=1, padding='same', activation='relu')(patch_inputs)  
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=2, padding='same')(x)

x = Conv2D(128, (3, 3), strides=1, padding='same', activation='relu')(x)  
x = Dropout(0.1)(x)
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=2, padding='same')(x)

x = Conv2D(128, (3, 3), strides=1, padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=2, padding='same')(x)

x = Conv2D(256, (3, 3), strides=1, padding='same', activation='relu')(x) 
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=2, padding='same')(x)

x = Conv2D(256, (3, 3), strides=1, padding='same', activation='relu')(x)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=2, padding='same')(x)

x = Conv2D(512, (3, 3), strides=1, padding='same', activation='relu')(x)  # Additional layer
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=2, padding='same')(x)

x = Flatten()(x)
patch_outputs = Dense(128, activation='relu')(x)
patch_outputs = Dropout(0.2)(patch_outputs)

patch_feature_model = Model(inputs=patch_inputs, outputs=patch_outputs)


I0000 00:00:1749114283.399220      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1749114283.400041      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [9]:
global_inputs = Input(shape=(img_size, img_size, 1))
x = Conv2D(64, (3, 3), strides=1, padding='same', activation='relu')(global_inputs) 
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=2, padding='same')(x)

x = Conv2D(128, (3, 3), strides=1, padding='same', activation='relu')(x) 
x = Dropout(0.1)(x)
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=2, padding='same')(x)

x = Conv2D(128, (3, 3), strides=1, padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=2, padding='same')(x)

x = Conv2D(256, (3, 3), strides=1, padding='same', activation='relu')(x)  
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=2, padding='same')(x)

x = Conv2D(256, (3, 3), strides=1, padding='same', activation='relu')(x)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=2, padding='same')(x)

x = Conv2D(512, (3, 3), strides=1, padding='same', activation='relu')(x)  # Additional layer
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=2, padding='same')(x)

x = Flatten()(x)
global_outputs = Dense(256, activation='relu')(x)
global_outputs = Dropout(0.2)(global_outputs)

global_feature_model = Model(inputs=global_inputs, outputs=global_outputs)

In [10]:
train_patch_features = patch_feature_model.predict(x_train_patches)
test_patch_features = patch_feature_model.predict(x_test_patches)

# Normalize patch features
patch_mean = np.mean(train_patch_features)
patch_std = np.std(train_patch_features)
train_patch_features = (train_patch_features - patch_mean) / patch_std
test_patch_features = (test_patch_features - patch_mean) / patch_std

# Reshape patch features to group by image
train_patch_features = train_patch_features.reshape(-1, num_patches_per_image, 128)
test_patch_features = test_patch_features.reshape(-1, num_patches_per_image, 128)

print(f"Grouped training patch features shape: {train_patch_features.shape}")
print(f"Grouped test patch features shape: {test_patch_features.shape}")

I0000 00:00:1749114288.958743      63 service.cc:148] XLA service 0x7a5d2c005920 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749114288.959579      63 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1749114288.959598      63 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1749114289.088737      63 cuda_dnn.cc:529] Loaded cuDNN version 90300


  50/4075 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step

I0000 00:00:1749114290.634356      63 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4075/4075 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step
488/488 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step
Grouped training patch features shape: (5216, 25, 128)
Grouped test patch features shape: (624, 25, 128)


In [11]:
train_global_features = global_feature_model.predict(x_train)
test_global_features = global_feature_model.predict(x_test)

# Normalize global features
global_mean = np.mean(train_global_features)
global_std = np.std(train_global_features)
train_global_features = (train_global_features - global_mean) / global_std
test_global_features = (test_global_features - global_mean) / global_std

print(f"Training global features shape: {train_global_features.shape}")
print(f"Test global features shape: {test_global_features.shape}")


163/163 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step
Training global features shape: (5216, 256)
Test global features shape: (624, 256)


In [12]:
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        self.dense = Dense(1, activation='tanh')

    def call(self, inputs):
        attention_scores = self.dense(inputs)  # (batch_size, num_patches, 1)
        attention_weights = tf.nn.softmax(attention_scores, axis=1)  # (batch_size, num_patches, 1)
        weighted_features = tf.reduce_sum(inputs * attention_weights, axis=1)  # (batch_size, patch_feature_dim)
        return weighted_features

In [13]:
def create_guiding_model(num_patches, patch_feature_dim, global_feature_dim):
    patch_inputs = Input(shape=(num_patches, patch_feature_dim))
    global_inputs = Input(shape=(global_feature_dim,))
    
    # Attention mechanism
    weighted_patch_features = AttentionLayer()(patch_inputs)  # (patch_feature_dim,)
    
    # Combine global and patch features
    combined_features = Concatenate()([weighted_patch_features, global_inputs])  # (patch_feature_dim + global_feature_dim,)
    
    # Deeper classifier with batch normalization
    x = Dense(256, activation='relu')(combined_features)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=[patch_inputs, global_inputs], outputs=outputs)
    return model

In [14]:
guiding_model = create_guiding_model(num_patches_per_image, 128, 256)

# Compute class weights to handle imbalance
num_pneumonia = np.sum(y_train == 0)
num_normal = np.sum(y_train == 1)
class_weight = {0: (num_normal / num_pneumonia), 1: 1.0}  # Weight Pneumonia class higher

guiding_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

guiding_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 25, 128)        │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_layer           │ (None, 128)            │            129 │ input_layer_2[0][0]    │
│ (AttentionLayer)          │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, 256)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 384)            │              0 │ attention_layer[0][0], │
│                           │                        │                │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 256)            │         98,560 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_12    │ (None, 256)            │          1,024 │ dense_3[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_10 (Dropout)      │ (None, 256)            │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 128)            │         32,896 │ dropout_10[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_13    │ (None, 128)            │            512 │ dense_4[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_11 (Dropout)      │ (None, 128)            │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 1)              │            129 │ dropout_11[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 133,250 (520.51 KB)

 Trainable params: 132,482 (517.51 KB)

 Non-trainable params: 768 (3.00 KB)

In [15]:
learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_accuracy',  # Monitor validation accuracy
    patience=3,
    verbose=1,
    factor=0.5,
    min_lr=0.00001
)

history = guiding_model.fit(
    [train_patch_features, train_global_features], y_train,
    batch_size=32,
    epochs=50,
    validation_data=([test_patch_features, test_global_features], y_test),
    callbacks=[learning_rate_reduction],
    class_weight=class_weight 
)


loss, accuracy = guiding_model.evaluate([test_patch_features, test_global_features], y_test)
print("Loss of the model is - ", loss)
print("Accuracy of the model is - ", accuracy * 100, "%")


predictions = (guiding_model.predict([test_patch_features, test_global_features]) > 0.5).astype(int).reshape(-1)
print("THE CLASSIFICATION REPORT OF WSI PIPELINE IS AS FOLLOWS:")
print(classification_report(y_test, predictions, target_names=['Pneumonia', 'Normal']))

Epoch 1/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.7911 - loss: 0.2134 - val_accuracy: 0.8622 - val_loss: 0.4199 - learning_rate: 0.0010
Epoch 2/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9207 - loss: 0.1001 - val_accuracy: 0.7788 - val_loss: 0.4933 - learning_rate: 0.0010
Epoch 3/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9437 - loss: 0.0785 - val_accuracy: 0.8077 - val_loss: 0.4877 - learning_rate: 0.0010
Epoch 4/50
153/163 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9401 - loss: 0.0758
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9402 - loss: 0.0759 - val_accuracy: 0.8109 - val_loss: 0.4752 - learning_rate: 0.0010
Epoch 5/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9511 - loss: 0.0610 - val_accuracy: 0.7308 - val_loss: 0.9580 - learning_rate: 5.0000e-04
Epoch 6/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9544 - loss: 0.063